In [139]:
import zipfile
from os import listdir
from os.path import isfile, join
from pathlib import Path

from abc import ABC, abstractmethod

from tqdm import tqdm

In [86]:
def instanceAllObjects(className, bytesArray, count, size):
    instanceList = []
    startByte = 0
    constructor = globals()[className]
    for o in range(0, count):
        instance = constructor(bytesArray[startByte:startByte + size])
        startByte += size
        instanceList.append(instance)
    return instanceList

In [161]:
def addZeros(bytesArray, finalSize):
    if(len(bytesArray) < finalSize):
        missingBytes = finalSize - len(bytesArray)
        zero = 0
        zeroByte = zero.to_bytes(1, 'little')
        bytesArray = bytesArray + (zeroByte * missingBytes)
    return bytesArray  

In [162]:
def fromListToBytes(objects, finalSize):
    objectsBytes =  bytearray()
    if len(objects) > 0:
        objectsBytesLists = list(map(lambda o: o.toBytes(), objects))
        objectsBytes = objectsBytesLists[0]
        for ob in objectsBytesLists[1:]:
            objectsBytes += ob
    return addZeros(objectsBytes, finalSize)    
    

In [87]:
class ObjectBytes(ABC):
    @abstractmethod
    def toBytes(self):
        pass

In [88]:
class GameObject(ObjectBytes):
    def __init__(self, objectBytes):
        self.xPosition = int.from_bytes(objectBytes[:4],"little", signed = False)
        self.yPosition = int.from_bytes(objectBytes[4:8],"little", signed = False)
        self.unUsed = int.from_bytes(objectBytes[8:10],"little", signed = False)
        self.objectWidth = objectBytes[10]
        self.objectHeight = objectBytes[11]
        self.objectFlags = int.from_bytes(objectBytes[12:16],"little", signed = False)
        self.childObjectFlags = int.from_bytes(objectBytes[16:20],"little", signed = False)
        self.extendedData = int.from_bytes(objectBytes[20:24],"little", signed = False)
        self.objectType = int.from_bytes(objectBytes[24:26],"little", signed = False)
        self.childObjectType = int.from_bytes(objectBytes[26:28],"little", signed = False)
        self.linkID = int.from_bytes(objectBytes[28:30],"little", signed = False)
        self.soundEffect = int.from_bytes(objectBytes[30:32],"little", signed = False)
    
    def toBytes(self):
        xBytes = self.xPosition.to_bytes(4, 'little')
        yBytes = self.yPosition.to_bytes(4, 'little')
        unUsedBytes = self.unUsed.to_bytes(2, 'little')
        width = self.objectWidth.to_bytes(1, 'little')
        height = self.objectHeight.to_bytes(1, 'little')
        flags = self.objectFlags.to_bytes(4, 'little')
        childObjectFlags = self.childObjectFlags.to_bytes(4, 'little')
        extendedData = self.extendedData.to_bytes(4, 'little')
        objectType = self.objectType.to_bytes(2, 'little')
        childObjectType = self.childObjectType.to_bytes(2, 'little')
        linkID = self.linkID.to_bytes(2, 'little')
        soundEffect = self.soundEffect.to_bytes(2, 'little')
        return xBytes + yBytes + unUsedBytes + width + height + flags + childObjectFlags + extendedData + objectType + childObjectType + linkID + soundEffect

In [89]:
class FreestandingSoundEffect(ObjectBytes):
    def __init__(self, soundBytes):
        self.effectType = soundBytes[0]
        self.xPosition = soundBytes[1]
        self.yPosition = soundBytes[2]
        self.padding = soundBytes[3]
        
    def toBytes(self):
        effectType = self.effectType.to_bytes(1, 'little')
        xPosition = self.xPosition.to_bytes(1, 'little')
        yPosition = self.yPosition.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        return effectType + xPosition + yPosition + padding

In [90]:
class SnakeNode(ObjectBytes):
    def __init__(self, snakeNodeBytes):
        self.index = int.from_bytes(snakeNodeBytes[:1],"little", signed = False) #parte da zero
        self.mistery = int.from_bytes(snakeNodeBytes[1:3],"little", signed = False)
        self.zerocento = int.from_bytes(snakeNodeBytes[3:5],"little", signed = False)
        self.unused = int.from_bytes(snakeNodeBytes[5:7],"little", signed = False) 
        
    def toBytes(self):
        index = self.index.to_bytes(2, 'little')
        mistery = self.mistery.to_bytes(2, 'little')
        zerocento = self.zerocento.to_bytes(2, 'little')
        unused = self.unused.to_bytes(2, 'little')
        return index + xPosition + mistery + zerocento + unused

In [91]:
class SnakeBlockTrack(ObjectBytes):
    def __init__(self, snakeBytes):
        self.snakeID = snakeBytes[0] #sarà 0 1 2 3 4?
        self.nodeCount = snakeBytes[1]
        self.always1 = snakeBytes[2]
        self.padding = snakeBytes[3]
        self.snakeNodes = instanceAllObjects('SnakeNode', snakesBytes[4:], self.snakeBlockCount, 8)
        
    def toBytes(self):
        snakeID = self.snakeID.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        always1 = self.always1.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        snakeNodes = fromListToBytes(self.snakeNodes, 964)
        return snakeID + nodeCount + always1 + padding + unused + snakeNodes  

In [92]:
class ClearPipeNode(ObjectBytes):
    def __init__(self, clearPipeNodesBytes):
        self.unknown011 = clearPipeNodesBytes[0]
        self.index = clearPipeNodesBytes[1]
        self.unKnowCoordinates = clearPipeNodesBytes[2]
        self.unKnowCoordinates2 = clearPipeNodesBytes[3]
        self.always2 = clearPipeNodesBytes[4]    
        
    def toBytes(self):
        unknown011 = self.unknown011.to_bytes(1, 'little')
        index = self.index.to_bytes(1, 'little')
        unKnowCoordinates = self.unKnowCoordinates.to_bytes(1, 'little')
        unKnowCoordinates2 = self.unKnowCoordinates2.to_bytes(1, 'little')
        always2 = self.always2.to_bytes(1, 'little')
        return unknown011 + index + unKnowCoordinates + unKnowCoordinates2 + always2

In [165]:
class ClearPipe(ObjectBytes):
    def __init__(self, clearPipeBytes):
        self.id = clearPipeBytes[0]
        self.nodeCount = clearPipeBytes[1]
        self.always1 = clearPipeBytes[2]
        self.padding = clearPipeBytes[3]
        self.clearPipeNodes = instanceAllObjects('ClearPipeNode', clearPipeBytes[4:], self.nodeCount, 8)
        
    def toBytes(self):
        pipeId = self.id.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        always1 = self.always1.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        clearPipeNodes = fromListToBytes(self.clearPipeNodes, 292)
        return pipeId + nodeCount + always1 + padding + unused + clearPipeNodes  

In [166]:
class PiranhaCreeperNode(ObjectBytes):
    def __init__(self, piranhaCreeperNodeBytes):
        self.always1 = piranhaCreeperNodeBytes[0] 
        self.valuesSeen = piranhaCreeperNodeBytes[1]
        self.always0_1 = piranhaCreeperNodeBytes[2]
        self.always0_2 = piranhaCreeperNodeBytes[3]
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        valuesSeen = self.valuesSeen.to_bytes(1, 'little')
        always0_1 = self.always0_1.to_bytes(1, 'little')
        always0_2 = self.always0_2.to_bytes(1, 'little')
        return always1 + valuesSeen + always0_1 + always0_2

In [95]:
class PiranhaCreeperTrack(ObjectBytes):
    def __init__(self, piranhaCreeperTrackBytes):
        self.always1 = piranhaCreeperTrackBytes[0] 
        self.idTrack = piranhaCreeperTrackBytes[1]
        self.nodeCount = piranhaCreeperTrackBytes[2]
        self.piranhaCreeperNodes = instanceAllObjects('PiranhaCreeperNode', piranhaCreeperTrackBytes[3:43], self.nodeCount, 4)
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        idTrack = self.idTrack.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        piranhaCreeperNodes = fromListToBytes(self.piranhaCreeperNodes, 84)
        return always1 + idTrack + nodeCount + piranhaCreeperNodes  

In [96]:
class ExpandingBlockNode(ObjectBytes):
    def __init__(self, expandingBlockNodeBytes):
        self.always1 = expandingBlockNodeBytes[0] 
        self.valuesSeen = expandingBlockNodeBytes[1]
        self.always0_1 = expandingBlockNodeBytes[2]
        self.always0_2 = expandingBlockNodeBytes[3]  
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        valuesSeen = self.valuesSeen.to_bytes(1, 'little')
        always0_1 = self.always0_1.to_bytes(1, 'little')
        always0_2 = self.always0_2.to_bytes(1, 'little')
        return always1 + valuesSeen + always0_1 + always0_2   

In [97]:
class ExpandingBlockTrack(ObjectBytes):
    def __init__(self, expandingBlockTrackBytes):
        self.always1 = expandingBlockNodeBytes[0] 
        self.idTrack = expandingBlockNodeBytes[1]
        self.nodeCount = expandingBlockNodeBytes[2]
        self.padding = expandingBlockNodeBytes[3]    
        self.expandingBlockNodes = instanceAllObjects('ExpandingBlockNode', expandingBlockNodeBytes[4:44], self.nodeCount, 4)
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        idTrack = self.idTrack.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        expandingBlockNodes = fromListToBytes(self.expandingBlockNodes, 44)
        return always1 + idTrack + nodeCount + padding + expandingBlockNodes      

In [98]:
class TrackBlockNode(ObjectBytes):
    def __init__(self, trackBlockNodeBytes):
        self.always1 = trackBlockNodeBytes[0]
        self.valuesSeen = trackBlockNodeBytes[1]
        self.always0_1 = trackBlockNodeBytes[2]
        self.always0_2 = trackBlockNodeBytes[3]
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        valuesSeen = self.valuesSeen.to_bytes(1, 'little')
        always0_1 = self.always0_1.to_bytes(1, 'little')
        always0_2 = self.always0_2.to_bytes(1, 'little')
        return always1 + valuesSeen + always0_1 + always0_2 

In [167]:
class TrackBlockTrack(ObjectBytes):
    def __init__(self, trackBlockTrackBytes):
        self.always1 = trackBlockTrackBytes[0] 
        self.idTrackBlock = trackBlockTrackBytes[1]
        self.nodeCount = trackBlockTrackBytes[2]
        self.unused = trackBlockTrackBytes[3]  
        self.trackBlockNodes = instanceAllObjects('TrackBlockNode', trackBlockTrackBytes[4:], self.nodeCount, 4)
        
    def toBytes(self):
        always1 = self.always1.to_bytes(1, 'little')
        idTrackBlock = self.idTrackBlock.to_bytes(1, 'little')
        nodeCount = self.nodeCount.to_bytes(1, 'little')
        unused = self.unused.to_bytes(1, 'little')
        trackBlockNodes = fromListToBytes(self.trackBlockNodes, 44)
        return always1 + idTrackBlock + nodeCount + unused + trackBlockNodes  

In [100]:
class Tile(ObjectBytes):
    def __init__(self, tileBytes):
        self.xPosition = tileBytes[0]
        self.yPosition = tileBytes[1]
        self.tileID = int.from_bytes(tileBytes[2:4],"little", signed = False) 
        
    def toBytes(self):
        xPosition = self.xPosition.to_bytes(1, 'little')
        yPosition = self.yPosition.to_bytes(1, 'little')
        tileID = self.tileID.to_bytes(2, 'little')
        return xPosition + yPosition + tileID      

In [101]:
class Rail(ObjectBytes):
    def __init__(self, railBytes):
        self.alwaysZero = int.from_bytes(railBytes[:1],"little", signed = False) 
        self.alwaysZeroOne = railBytes[1]
        self.xPosition = railBytes[2]
        self.yPosition = railBytes[3]
        self.unknown014 = railBytes[4]
        self.unknown1 = int.from_bytes(railBytes[5:7],"little", signed = False) 
        self.unknown2 = int.from_bytes(railBytes[7:9],"little", signed = False) 
        self.unknown3 = int.from_bytes(railBytes[9:11],"little", signed = False) 
    def toBytes(self):
        alwaysZero = self.alwaysZero.to_bytes(2, 'little')
        alwaysZeroOne = self.alwaysZeroOne.to_bytes(1, 'little')
        xPosition = self.xPosition.to_bytes(1, 'little')
        yPosition = self.yPosition.to_bytes(1, 'little')
        unknown014 = self.unknown014.to_bytes(1, 'little')
        unknown1 = self.unknown1.to_bytes(2, 'little')
        unknown2 = self.unknown2.to_bytes(2, 'little')
        unknown3 = self.unknown3.to_bytes(2, 'little')
        return alwaysZero + alwaysZeroOne + xPosition + yPosition + unknown014 + unknown1 + unknown2 + unknown3

In [102]:
class Icicle(ObjectBytes):
    def __init__(self, icicleBytes):
        self.xPosition = icicleBytes[0]
        self.yPosition = icicleBytes[1]
        self.zeroOne = icicleBytes[2]
        self.padding = icicleBytes[3]
    def toBytes(self):
        xPosition = self.xPosition.to_bytes(1, 'little')
        yPosition = self.yPosition.to_bytes(1, 'little')
        zeroOne = self.zeroOne.to_bytes(1, 'little')
        padding = self.padding.to_bytes(1, 'little')
        return xPosition + yPosition + zeroOne + padding

In [131]:
class LevelArea(ObjectBytes):
    def __init__(self, levelBytes):
        self.courseTheme = levelBytes[0]
        self.autoScrollType = levelBytes[1]
        self.unknowHorientation = levelBytes[2]
        self.levelHorientation = levelBytes[3]
        self.lavaWaterHeight = levelBytes[4]
        self.lavaWaterMode = levelBytes[5]
        self.lavaWaterSpeed = levelBytes[6]
        self.minimunLavaWaterHeight = levelBytes[7]
        self.rightBoundary = int.from_bytes(levelBytes[8:12],"little", signed = False)
        self.topBoundary = int.from_bytes(levelBytes[12:16],"little", signed = False)
        self.leftBoundary = int.from_bytes(levelBytes[16:20],"little", signed = False)
        self.bottomBoundary = int.from_bytes(levelBytes[20:24],"little", signed = False)
        self.bitField = int.from_bytes(levelBytes[24:28],"little", signed = False)
        self.objectCount = int.from_bytes(levelBytes[28:32],"little", signed = False)
        self.freeStandingSoundEffectCount = int.from_bytes(levelBytes[32:36],"little", signed = False)
        self.snakeBlockCount = int.from_bytes(levelBytes[36:40],"little", signed = False)
        self.clearPipeCount = int.from_bytes(levelBytes[40:44],"little", signed = False)
        self.piranhaCreeperCount = int.from_bytes(levelBytes[44:48],"little", signed = False)
        self.expandingBlockCount = int.from_bytes(levelBytes[48:52],"little", signed = False)
        self.trackBlockCount = int.from_bytes(levelBytes[52:56],"little", signed = False)
        self.alwaysZero = int.from_bytes(levelBytes[56:60],"little", signed = False)
        self.tileCount = int.from_bytes(levelBytes[60:64],"little", signed = False)
        self.railCount = int.from_bytes(levelBytes[64:68],"little", signed = False)
        self.icicleCount = int.from_bytes(levelBytes[68:72],"little", signed = False)
        
        
        
        objectsBytes = levelBytes[72:83272]
        self.objects = instanceAllObjects('GameObject', objectsBytes, self.objectCount, 32)
        
        
        soundEffectsBytes = levelBytes[83272:84472]
        self.soundEffects = instanceAllObjects('FreestandingSoundEffect', soundEffectsBytes, self.freeStandingSoundEffectCount, 4)
        
        
        snakesBytes = levelBytes[84472:89292]
        self.snakes = instanceAllObjects('SnakeBlockTrack', snakesBytes, self.snakeBlockCount, 964)
        
        
        clearPipeBytes = levelBytes[89292:147692]
        self.clearPipes = instanceAllObjects('ClearPipe', clearPipeBytes, self.clearPipeCount, 292)
        
        creeperBytes = levelBytes[147692:148532]
        self.creepers = instanceAllObjects('PiranhaCreeperTrack', creeperBytes, self.piranhaCreeperCount, 84)
        
        expandingBlockBytes = levelBytes[148532:148972]
        self.expandingBlocks = instanceAllObjects('ExpandingBlockTrack', expandingBlockBytes, self.expandingBlockCount, 44)
        
        trackBlockBytes = levelBytes[148972:149412]
        self.trackBlocks = instanceAllObjects('TrackBlockTrack', trackBlockBytes, self.trackBlockCount, 44)
        
        tilesBytes = levelBytes[149412:165412]
        self.tiles = instanceAllObjects('Tile', tilesBytes, self.tileCount, 4)
        
        railBytes = levelBytes[165412:183412]
        self.rails = instanceAllObjects('Rail', railBytes, self.railCount, 12)
        
        iCicleBytes = levelBytes[183412:184612]
        self.icicles = instanceAllObjects('Icicle', iCicleBytes, self.icicleCount, 4)
        
    def toBytes(self):
        courseTheme = self.courseTheme.to_bytes(1, 'little')
        autoScrollType = self.autoScrollType.to_bytes(1, 'little')
        unknowHorientation = self.unknowHorientation.to_bytes(1, 'little')
        levelHorientation = self.levelHorientation.to_bytes(1, 'little')
        lavaWaterHeight = self.lavaWaterHeight.to_bytes(1, 'little')
        lavaWaterMode = self.lavaWaterMode.to_bytes(1, 'little')
        lavaWaterSpeed = self.lavaWaterSpeed.to_bytes(1, 'little')
        minimunLavaWaterHeight = self.minimunLavaWaterHeight.to_bytes(1, 'little')
        rightBoundary = self.rightBoundary.to_bytes(4, 'little')
        topBoundary = self.topBoundary.to_bytes(4, 'little')
        leftBoundary = self.leftBoundary.to_bytes(4, 'little')
        rightBoundary = self.rightBoundary.to_bytes(4, 'little')
        bitField = self.bitField.to_bytes(4, 'little')
        objectCount = self.objectCount.to_bytes(4, 'little')
        freeStandingSoundEffectCount = self.freeStandingSoundEffectCount.to_bytes(4, 'little')
        snakeBlockCount = self.snakeBlockCount.to_bytes(4, 'little')
        clearPipeCount = self.clearPipeCount.to_bytes(4, 'little')
        piranhaCreeperCount = self.piranhaCreeperCount.to_bytes(4, 'little')
        expandingBlockCount = self.expandingBlockCount.to_bytes(4, 'little')
        trackBlockCount = self.trackBlockCount.to_bytes(4, 'little')
        alwaysZero = self.alwaysZero.to_bytes(4, 'little')
        tileCount = self.tileCount.to_bytes(4, 'little')
        railCount = self.railCount.to_bytes(4, 'little')
        icicleCount = self.icicleCount.to_bytes(4, 'little')
        returnBytes = courseTheme + autoScrollType + unknowHorientation + levelHorientation + lavaWaterHeight + lavaWaterMode + lavaWaterSpeed + minimunLavaWaterHeight  \
               + rightBoundary + topBoundary + leftBoundary + bottomBoundary + bitField + objectCount + freeStandingSoundEffectCount + snakeBlockCount + clearPipeCount \
               + piranhaCreeperCount + expandingBlockCount + trackBlockCount + alwaysZero + tileCount + railCount + icicleCount
        objectsBytes = fromListToBytes(self.objects, 83200)
        soundEffectsBytes = fromListToBytes(self.soundEffects, 1200)
        snakesBytes = fromListToBytes(self.snakes, 4820)
        clearPipeBytes = fromListToBytes(self.clearPipes, 58400)
        creeperBytes = fromListToBytes(self.creepers, 840)
        expandingBlockBytes = fromListToBytes(self.expandingBlocks, 440)
        trackBlockBytes = fromListToBytes(self.trackBlocks, 440)
        tilesBytes = fromListToBytes(self.tiles, 16000)
        railBytes = fromListToBytes(self.rails, 18000)
        iCicleBytes = fromListToBytes(self.icicles, 1200)
        return returnBytes +  objectsBytes + soundEffectsBytes + snakesBytes + clearPipeBytes + creeperBytes + expandingBlockBytes + trackBlockBytes + tilesBytes + railBytes + iCicleBytes
        

In [199]:
class Header(ObjectBytes):
    def __init__(self, headerBytes):
        self.startY = headerBytes[0]
        self.endY = headerBytes[1]
        self.goalXx100 = int.from_bytes(headerBytes[2:4],"little", signed = False)
        self.timeLimit = int.from_bytes(headerBytes[4:6],"little", signed = False)
        self.targetAmountClearCondition = int.from_bytes(headerBytes[6:8],"little", signed = False)
        self.year = int.from_bytes(headerBytes[8:10],"little", signed = False)
        self.month = headerBytes[10]
        self.day = headerBytes[11]
        self.hour = headerBytes[12]
        self.minute = headerBytes[13]
        self.unknow = headerBytes[14]
        self.clearConditionType = headerBytes[15]
        self.clearConditionObject = int.from_bytes(headerBytes[16:20],"little", signed = False)
        self.gameVersion = int.from_bytes(headerBytes[20:24],"little", signed = False)
        self.levelFlags = int.from_bytes(headerBytes[24:28],"little", signed = False)
        self.clearCheckTries = int.from_bytes(headerBytes[28:32],"little", signed = False)
        self.clearCheckTime = int.from_bytes(headerBytes[32:36],"little", signed = False)
        self.creationID = int.from_bytes(headerBytes[36:40],"little", signed = False)
        self.uploadID = headerBytes[40:48]
        self.gameVersionCheck = headerBytes[48:52]
        self.padding = headerBytes[52:240]
        self.unknow2 = headerBytes[240]
        self.gameStyle = headerBytes[241:244]
        self.courseName = headerBytes[244:310]#wchar16
        self.courseDescription = headerBytes[310:378]
        
    def toBytes(self):
        startY = self.startY.to_bytes(1, 'little')
        endY = self.endY.to_bytes(1, 'little')
        goalXx100 = self.goalXx100.to_bytes(2, 'little')
        timeLimit = self.timeLimit.to_bytes(2, 'little')
        targetAmountClearCondition = self.targetAmountClearCondition.to_bytes(2, 'little')
        year = self.year.to_bytes(2, 'little')
        month = self.month.to_bytes(1, 'little')
        day = self.day.to_bytes(1, 'little')
        hour = self.hour.to_bytes(1, 'little')
        minute = self.minute.to_bytes(1, 'little')   
        unknow = self.unknow.to_bytes(1, 'little') 
        clearConditionType = self.clearConditionType.to_bytes(1, 'little') 
        clearConditionObject = self.clearConditionObject.to_bytes(4, 'little') 
        gameVersion = self.gameVersion.to_bytes(4, 'little')
        levelFlags = self.levelFlags.to_bytes(4, 'little')
        clearCheckTries = self.clearCheckTries.to_bytes(4, 'little')
        clearCheckTime = self.clearCheckTime.to_bytes(4, 'little')
        creationID = self.creationID.to_bytes(4, 'little')
        unknow2 = self.unknow2.to_bytes(1, 'little')
        
        return startY + endY + goalXx100 + timeLimit + targetAmountClearCondition + year + month + day + hour + minute + unknow + clearConditionType \
               + clearConditionObject + gameVersion + levelFlags + clearCheckTries + clearCheckTime + creationID + self.uploadID \
               + self.gameVersionCheck + self.padding + unknow2 + self.gameStyle + self.courseName + self.courseDescription

In [197]:
class MarioMaker2File(ObjectBytes):
    def __init__(self, fileBytes):
        self.gameVersion = fileBytes[:4]
        self.gameVersion2 = fileBytes[4:6]
        self.padding = fileBytes[6:8]
        self.CRC32 = fileBytes[8:12]
        self.SCDL = fileBytes[12:16]
        self.header = Header(fileBytes[16:528])
        self.mainLevelArea = LevelArea(fileBytes[528:188656])
        self.subLevelArea = LevelArea(fileBytes[188656:376784])
        self.cryptoCfg = fileBytes[376784:]#non ne sono sicuro
        
    def toBytes(self):
        return self.gameVersion + self.gameVersion2 + self.padding + self.CRC32 + self.SCDL + addZeros(self.header.toBytes(), 512) + addZeros(self.mainLevelArea.toBytes(), 188128) + addZeros(self.subLevelArea.toBytes(), 188128) + self.cryptoCfg

In [20]:
levelFolder = Path("levels/")

In [21]:
levelsDownloaded = [f for f in listdir(levelFolder) if isfile(join(levelFolder, f))]

In [22]:
archive = zipfile.ZipFile(join(levelFolder, '007_-_bond_mario_bond__2958.zip'), 'r')
leveldata = archive.read('course_data_000.bcd')

In [172]:
len(leveldata)

376832

In [24]:
with open('tools/tmp/course_data_000.bcd', 'wb') as f:
    f.write(leveldata)

In [25]:
!.\tools\smm2dec.exe -h .\tools\tmp\course_data_000.bcd .\tools\tmp\course_data_dec.bcd

Decrypting course .\tools\tmp\course_data_000.bcd to .\tools\tmp\course_data_dec.bcd...
Done!


In [26]:
with open('tools/tmp/course_data_dec.bcd', 'rb') as f:
    leveldata = f.read()

In [27]:
leveldata[15215]

0

In [200]:
testM = MarioMaker2File(leveldata)
len(testM.toBytes())

376832

In [204]:
a = testM.toBytes()
print(len(a))
for idx, b in enumerate(leveldata):
    if a[idx] != b:
        print('porco dio')
        print(idx)
        break

376832
porco dio
165943


In [192]:
AESIV = leveldata[:16]  #array
randomiserState = leveldata[17:33]
AESCMAC = leveldata[34:50]


gameVersion = leveldata[:4]
gameVersion2 = leveldata[4:6]
padding = leveldata[6:8]
CRC32 = leveldata[8:12]
SCDL = leveldata[12:16]
header = leveldata[16:528]
mainLevelArea = leveldata[528:188656]
subLevelArea = leveldata[188656:376784]
CryptoCfg = leveldata[376784:]#non ne sono sicuro


In [29]:
CryptoCfg

b"\xf8T\x028'\xae\x05N\x07\xf9 \xeb\x84\xe8n\x1c'\xae\x05N\xf8T\x028\x84\xe8n\x1c\x07\xf9 \xeb\xe0\x18<=\x8aZ\xf5F0\xcd\xc6r\n<\xe1\xaf"

In [48]:
startY = header[0]
endY = header[1]
goalXx100 = header[2:4]
timeLimit = header[4:6]
targetAmountClearCondition = header[6:8]
year = header[8:10]
month = header[10]
day = header[11]
hour = header[12]
minute = header[13]
unknow = header[14]
clearConditionType = header[15]
clearConditionObject = header[16:20]
unknow4 = header[20:24]
levelFlags = header[24:28]
clearCheckTime = header[28:32]
randomValue = header[32:36]
courseWorld = header[36:40]

unknow2 = header[240]
gameStyle = header[241:244]
courseName = header[244:310]#wchar16
courseDescription = header[310:378]

In [49]:
courseTheme = mainLevelArea[0]
autoScrollType = mainLevelArea[1]
unknowHorientation = mainLevelArea[2]
levelHorientation = mainLevelArea[3]
lavaWaterHeight = mainLevelArea[4]
lavaWaterMode = mainLevelArea[5]
lavaWaterSpeed = mainLevelArea[6]
minimunLavaWaterHeight = mainLevelArea[7]
rightBoundary = mainLevelArea[8:12]
topBoundary = mainLevelArea[12:16]
leftBoundary = mainLevelArea[16:20]
bottomBoundary = mainLevelArea[20:24]
bitField = mainLevelArea[24:28]
objectCount = mainLevelArea[28:32]
freeStandingSoundEffectCount = mainLevelArea[32:36]
snakeBlockCount = mainLevelArea[36:40]
clearPipeCount = mainLevelArea[40:44]
piranhaCreeperCount = mainLevelArea[44:48]
expandingBlockCount = mainLevelArea[48:52]
trackBlockCount = mainLevelArea[52:56]
alwaysZero = mainLevelArea[56:60]
tileCount = mainLevelArea[60:64]
railCount = mainLevelArea[64:68]
icicleCount = mainLevelArea[68:72]
objects = mainLevelArea[72:83272]
soundEffects = mainLevelArea[83272:84472]
snake = mainLevelArea[84472:89292]
clearPipe = mainLevelArea[89292:147692]
creeper = mainLevelArea[147692:148532]
expandingBlock = mainLevelArea[148532:148972]
trackBlock = mainLevelArea[148972:149412]
tiles = mainLevelArea[149412:165412]
rail = mainLevelArea[165412:183412]
iCicle = mainLevelArea[183412:184612]

In [163]:
main = LevelArea(mainLevelArea)

In [164]:
len(main.toBytes())

184612

In [107]:
courseTheme

5

In [15]:
int.from_bytes(tileCount,"little", signed = False)

1128

In [16]:
gameVersion2

b'\x10\x00'

In [17]:
month

7

In [18]:
courseName.decode("utf-16")

'007 - Bond, Mario Bond ♪\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [19]:
courseDescription.decode("utf-16")

'Music and Play!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [20]:
gameStyle.decode("utf-8")

'M3\x00'